In [1]:
import sys
sys.path.append('..')

In [2]:
from typing import Generic, TypeVar, Callable

from operator import truediv

from datatypes import datatype, match

In [3]:
A = TypeVar('A')

In [7]:
@datatype
class Result(Generic[A]):
    Ok    : (A,)
    Error : (Exception,)

    @classmethod
    def attempt(cls, f : Callable, *args, **kwargs):
        try:
            return cls.Ok(f(*args, **kwargs))
        except Exception as e:
            return cls.Error(e)

    def map(self, f):
        return match(self, {
            Result.Ok   : lambda a: Result.Ok(f(a)),
            Result.Error: lambda e: Error(e),
        })

    def bind(self, f):
        return match(self, {
            Result.Ok   : lambda a: f(a),
            Result.Error: lambda e: Error(e),
        })

    def then(self, f):
        result = self.bind(f)
        
        return result if isinstance(result, Result) else Result.Ok(result)
    
    __rlshift__ = __rshift__ = then


attempt = Result.attempt

In [8]:
attempt(truediv, 2, 3)

Ok(0.6666666666666666)

In [11]:
attempt(truediv, 2, 0)

Error(division by zero)

In [17]:
(
    attempt(truediv, 2, 3)
    >> (lambda x: x*2)
    >> (lambda x: attempt(truediv, x, 2))
)

Ok(0.6666666666666666)